In [8]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
file_dir = "./Retinal Fundus Images"

In [4]:
os.listdir(file_dir)

['test', 'train', 'val']

In [9]:
def rgb_to_greyscale(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

In [23]:
#since test-train-val directories are already created, going to create generators

size = 500
samples = 10

train_datagen = ImageDataGenerator(
    rescale = 1.0/255.0,
    preprocessing_function = rgb_to_greyscale
    # rotation_range = 20,
    # width_shift_range = 0.02,
    # height_shift_range = 0.02,
    # shear_range = 0.001,
    # zoom_range = 0.02,
    # horizontal_flip = True,
    # vertical_flip = True
)
train_generator = train_datagen.flow_from_directory(
    directory = os.path.join(file_dir, 'train'),
    batch_size = samples,
    class_mode = 'categorical',
    target_size = (size, size),
    shuffle = True
)

validation_datagen = ImageDataGenerator(rescale = 1.0/255.0, preprocessing_function = rgb_to_greyscale)
validation_generator = validation_datagen.flow_from_directory(
    directory = os.path.join(file_dir, 'val'),
    batch_size = samples,
    class_mode = 'categorical',
    target_size = (size,size)
)

test_datagen = ImageDataGenerator(rescale = 1.0/255.0, preprocessing_function = rgb_to_greyscale)
test_generator = validation_datagen.flow_from_directory(
    directory = os.path.join(file_dir, 'test'),
    batch_size = samples,
    class_mode = 'categorical',
    target_size = (size,size)
)

Found 20077 images belonging to 11 classes.
Found 433 images belonging to 11 classes.
Found 1236 images belonging to 11 classes.


In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', padding = 'same', input_shape = (size, size, 3)),
    tf.keras.layers.MaxPooling2D((4,4)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.MaxPooling2D((3,3)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.MaxPooling2D((3,3)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(11, activation = 'softmax')

])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

print(model.summary())

with tf.device('GPU:0'):
    history = model.fit(
        train_generator,
        validation_data =  validation_generator,
        epochs = 25,
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 4)]
    )

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 500, 500, 32)      1568      
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 125, 125, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 41, 41, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 41, 41, 64)        36928     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 13, 13, 64)      

In [26]:
#model.save_weights("bad_model_1")